In [1]:
# Initiate game
# generate n_players suits where n_players is currently 5
# generate n_cards per suit
# shuffle cards


# Player
# Once the trade is executed, register the trade on their own ledger
## information recorded
# Functionalities reveal private information
# Compute private and perceived probabilities
# generate insights
# plot risks


# MarketMakers
# ability to set bid and ask price
# arbitage alert


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Functions

In [3]:
def shuffle(deck):
    shuffled_deck = []
    for i in range(len(deck)):
        idx = np.random.randint(0, len(deck))
        shuffled_deck.append(deck.pop(idx))
    return shuffled_deck

In [4]:
def generate_cards(n_cards, suits):
    deck = []
    for suit in suits:
        cards = [suit + ' '+ str(num) for num in np.linspace(1, n_cards, n_cards).astype('int')]
        deck.extend(cards)
    return deck

In [5]:
def distribute(deck, n_cards):
    private_information = []
    for idx in range(n_cards):
        private_information.append(deck.pop(np.random.randint(0, len(deck))))
    return private_information
    

In [36]:
def mergePrivateAndPublic(privateCards, revealedCards):
    temp = privateCards.copy()
    temp.extend(revealedCards)
    return temp

# to calculated private prob
# first merge public and private

# to calculated public prob
# use public cards only

# Note once a private card has been revealed, they should be added to the revealed public card 
def calculateProbability(cardsExcluded, n_cards_per_suit, suits):
    d = {}
    for s in suits:
        d[s] = n_cards_per_suit
        
    for c in cardsExcluded:
        suit, _ = c.split(' ')
        d[suit] -= 1
    # convert to prob
    return [round(d[key]/n_cards_per_suit, 3) for key in suits] 
    

# Classes

In [106]:
class PublicBoard:
    suits = ['S', 'H', 'C', 'D']
    n_cards_per_suit = 13
    def __init__(self):
        self.revealedCards = []
        d = {}
        for key in ['Public Prob ' + str(suit) for suit in suits]:
            d[key] = [1.0]
        self.publicInformation = pd.DataFrame.from_dict(d, orient='columns')
    
    def updateProbability(self, num):
        # update public probability
        self.publicInformation.loc[num] = calculateProbability(self.revealedCards, n_cards_per_suit, suits)
    
    def revealedFromPublicDeck(self, deck):
        revealedCard = deck.pop()
        self.revealedCards.append(revealedCard)
        return revealedCard
        
    # add plotting tools here

In [102]:
class Player:
    suits = ['S', 'H', 'C', 'D']
    n_cards_per_suit
    def __init__(self, privateCards, name):
#          self.ledger = pd.DataFrame()
#         self.inventories = pd.DataFrame(columns= suits.copy() + ['Cash'])
        self.name = name
        self.privateCards = privateCards
        
        d = {}
        for key in ['Private Prob ' + str(suit) for suit in suits]:
            d[key] = [1.0]
        self.privateInformation = pd.DataFrame.from_dict(d, orient='columns')
        
    def updateProbability(self, num, revealedCards):
        # update private probability
        self.privateInformation.loc[num] = calculateProbability(\
                                            mergePrivateAndPublic(self.privateCards, revealedCards), n_cards_per_suit, suits)
    
    def revealPrivateCard(self, cardToBeRevealed):
        if cardToBeRevealed in self.privateInfo:
            idx = self.privateInfo.idx(cardToBeRevealed)
            cardRevealed = self.privateCards.pop(idx)
            return cardRevealed
        else:
            pass # raise error
        
        

# Initiate Game

In [50]:
# number of market makers
n_market_makers = 4 # this is always fixed
suits = ['S', 'C', 'D', 'H'] 
# number of spectator
n_spectator = 1 # either 0 or 1

# total number of players
n_total_players = n_market_makers + n_spectator

# number of cards per suit
n_cards_per_suit = 13
# n_total_cards = n_cards*n_market_makers

# maximum bid-ask spread
max_bid_ask_spread = 5


In [97]:
###### Inititate Game ######
# 1. generate cards
deck = generate_cards(n_cards_per_suit, suits)
print('{} Cards generated.'.format(len(deck)))

# 2. shuffle deck
deck = shuffle(deck)
print('Cards shuffled.')

# 3. distribute cards
if n_total_players == 4:
    # distribute 4 cards for MM
    privateInfoMM1, privateInfoMM1,\
    privateInfoMM1, privateInfoMM1 = [distribute(deck, 4) for _ in range(4)]
elif n_total_players == 5:
    # distribute 5 cards for MM
    privateInfoMM1, privateInfoMM2,\
    privateInfoMM3, privateInfoMM4 = [distribute(deck, 5) for _ in range(4)]
    
    # 3 cards for spectator
    privateInfoSpec = distribute(deck, 3)
print('Players obtained private information.')
print('{} Cards left'.format(len(deck)))

52 Cards generated.
Cards shuffled.
Players obtained private information.
29 Cards left


# 2. Create Players and Public Board
1. Created players and public boad
2. Simulated the action of revealing cards from the deck. Updated the public probabilities and the perceived probabilities simultaneously

TODO:
set bid ask price
execute trade: buy or sell
update inventories

In [115]:
# create players
p1 = Player(privateInfoMM1, suits[0])

p2 = Player(privateInfoMM2, suits[1])

p3 = Player(privateInfoMM3, suits[2])

p4 = Player(privateInfoMM4, suits[3])

p5 = Player(privateInfoSpec, 'Spectator')

# for convenience
players = [p1, p2, p3, p4 ,p5]

In [116]:
# create public board
pb = PublicBoard()

In [117]:
#  example of card reveal, copy deck to avoid regenerating deck of cards
tempDeck = deck.copy()
num = 0
print(len(tempDeck))

# reveal 10 cards consecutively
for _ in range(10):
    num += 1
    pb.revealedFromPublicDeck(tempDeck)
    print(pb.revealedCards)
    pb.updateProbability(num)
    for player in players:
        player.updateProbability(num, pb.revealedCards)
    
# display public information
pb.publicInformation
print(len(tempDeck))

29
['H 13']
['H 13', 'H 3']
['H 13', 'H 3', 'H 7']
['H 13', 'H 3', 'H 7', 'D 3']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4', 'D 13']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4', 'D 13', 'S 9']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4', 'D 13', 'S 9', 'H 4']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4', 'D 13', 'S 9', 'H 4', 'C 2']
['H 13', 'H 3', 'H 7', 'D 3', 'D 4', 'D 13', 'S 9', 'H 4', 'C 2', 'D 7']
19


In [123]:
for player in players:
    player.updateProbability(1, pb.revealedCards)
    print(player.privateCards)
    print('########## Public Info ##########')
    print(pb.publicInformation.loc[10])
    print('########## ' + player.name + ' Private Info ##########')
    print(player.privateInformation.loc[10])



['S 4', 'D 10', 'H 5', 'C 5', 'H 2']
########## Public Info ##########
Public Prob S    0.923
Public Prob C    0.923
Public Prob D    0.692
Public Prob H    0.692
Name: 10, dtype: float64
########## S Private Info ##########
Private Prob S    0.846
Private Prob C    0.846
Private Prob D    0.615
Private Prob H    0.538
Name: 10, dtype: float64
['D 2', 'S 1', 'C 12', 'H 1', 'D 11']
########## Public Info ##########
Public Prob S    0.923
Public Prob C    0.923
Public Prob D    0.692
Public Prob H    0.692
Name: 10, dtype: float64
########## C Private Info ##########
Private Prob S    0.846
Private Prob C    0.846
Private Prob D    0.538
Private Prob H    0.615
Name: 10, dtype: float64
['D 5', 'D 6', 'S 2', 'D 1', 'S 13']
########## Public Info ##########
Public Prob S    0.923
Public Prob C    0.923
Public Prob D    0.692
Public Prob H    0.692
Name: 10, dtype: float64
########## D Private Info ##########
Private Prob S    0.769
Private Prob C    0.923
Private Prob D    0.462
Private Pr

In [62]:
# Player
# Once the trade is executed, register the trade on their own ledger
## information recorded
# Functionalities reveal private information
# Compute private and perceived probabilities
# generate insights
# plot risks